# Document Clustering and Topic Modeling

*In* this project, I use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [17]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# code: 4/uwEfuaFnozf-xDHs09yS6tqzHqw-e1DPqPrqGLl38zetEYfrDHeWhpo

In [18]:
# https://drive.google.com/open?id=192JMR7SIqoa14vrs7Z9BXO3iK89pimJL
file = drive.CreateFile({'id':'192JMR7SIqoa14vrs7Z9BXO3iK89pimJL'}) # replace the id with id of file you want to access
file.GetContentFile('data.tsv')  

# Part 1: Load Data

In [19]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import nltk
import gensim
# REGULAR EXPRESSION
import re
import os

from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# Load data into dataframe
df = pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [21]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [22]:
# Remove missing value
df.review_body.dropna(inplace=True)

In [23]:
# use the first 1000 data as our training data
data = df.loc[:1000, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [24]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 179 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [25]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string 
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# tokenization without stemming
def tokenization(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [26]:
# tokenization and stemming
# lemmatization
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'watch',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'i',
 'wear',
 'dainti']

Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [27]:
# 1. do tokenization and stemming for all the documents
# 2. also just do tokenization for all the documents
# the goal is to create a mapping from stemmed words to original tokenized words for result interpretation.
docs_stemmed = []
docs_tokenized = []
for i in data:
    tokenized_and_stemmed_results = tokenization_and_stemming(i)
    docs_stemmed.extend(tokenized_and_stemmed_results)
    
    tokenized_results = tokenization(i)
    docs_tokenized.extend(tokenized_results)

In [28]:
# create a mapping from stemmed words to original words
vocab_frame_dict = {docs_stemmed[x]:docs_tokenized[x] for x in range(len(docs_stemmed))}

In [29]:
vocab_frame_dict

{'absolut': 'absolutely',
 'love': 'loved',
 'watch': 'watch',
 'get': 'get',
 'compliment': 'compliments',
 'almost': 'almost',
 'everi': 'every',
 'time': 'time',
 'i': 'i',
 'wear': 'wear',
 'dainti': 'dainty',
 'keep': 'keep',
 'wonder': 'wonderful',
 'scratch': 'scratches',
 'it': 'it',
 'work': 'work',
 'well': 'well',
 'howev': 'however',
 'found': 'found',
 'cheaper': 'cheaper',
 'price': 'price',
 'place': 'place',
 'make': 'make',
 'purchas': 'purchased',
 'beauti': 'beautiful',
 'face': 'face',
 'the': 'the',
 'band': 'band',
 'look': 'looks',
 'nice': 'nice',
 'around': 'around',
 'link': 'links',
 'squeaki': 'squeaky',
 'cheapo': 'cheapo',
 'nois': 'noise',
 'swing': 'swing',
 'back': 'back',
 'forth': 'forth',
 'wrist': 'wrist',
 'embarrass': 'embarrassing',
 'front': 'front',
 'enthusiast': 'enthusiasts',
 'nake': 'naked',
 'eye': 'eyes',
 'afar': 'afar',
 'ca': 'ca',
 "n't": "n't",
 'tell': 'tell',
 'cheap': 'cheap',
 'fold': 'folds',
 'polish': 'polishing',
 'brush': '

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

***example:***

document1: "I love You"

document 2: "You love love me"

document1: tf-idf [1, 0.5, 0.5, 0];  document2: tf-idf [0, 1, 0.5, 1]  



In [30]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 1000 reviews and 245 terms.


In [31]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming>,
 'use_idf': True,
 'vocabulary': None}

Save the terms identified by TF-IDF.

In [32]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [33]:
# print out words
tf_selected_words

["'m",
 "'s",
 'abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'br',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi

# Calculate Document Similarity

In [34]:
# use cosine similarity to check the similarity for two documents
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(tfidf_matrix)
print (cos_matrix)

[[1.         0.42500259 0.         ... 0.         0.0400497  0.        ]
 [0.42500259 1.         0.         ... 0.         0.09423401 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.0400497  0.09423401 0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


# Part 4: K-means clustering

In [35]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [36]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].product_title, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [37]:
frame.head(10)

,review,cluster
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",2
1,Kenneth Cole New York Women's KC4944 Automatic...,2
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,4
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,4
4,Orient ER27009B Men's Symphony Automatic Stain...,1
5,Casio Men's GW-9400BJ-1JF G-Shock Master of G ...,2
6,Fossil Women's ES3851 Urban Traveler Multifunc...,1
7,INFANTRY Mens Night Vision Analog Quartz Wrist...,4
8,G-Shock Men's Grey Sport Watch,1
9,Heiden Quad Watch Winder in Black Leather,1


In [38]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
4,455
1,283
2,119
3,77
0,66


In [39]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(vocab_frame_dict[tf_selected_words[ind]])
        print (vocab_frame_dict[tf_selected_words[ind]] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

<Document clustering result by K-means>
Cluster 0 words:nice,watch,price,looks,really,simple,
Cluster 0 reviews (66 reviews): 
Guess U13630G1 Men's day and date Gunmetal dial Gunmetal tone bracelet, Casio Men's MRW200H-7EV Sport Resin Watch, Casio G-shock Mudman Multiband6 Japanese Model [ Gw-9300-1jf ], Stuhrling Original Men's 395.33U16 Aquadiver Regatta Analog Swiss Quartz Stainless Steel Link Bracelet Watch, Invicta Men's Bolt 12298, Invicta Men's 3332 Force Collection Stainless Steel Left-Handed Watch with Black Leather Band, Timex Ironman Rugged 30 Full-Size Watch, Nautica Men's A16577G BFD 105 Chrono Analog Display Analog Quartz Black Watch, Invicta Women's 16267 ANGEL Analog Display Japanese Quartz Grey Watch, U.S. Polo Assn. Classic Men's US4023 Watch with Black Leather Band, Men's Gold-Tone Casio World Time Stainless Steel Watch A500WGA-1, Daniel Wellington Women's 0506DW Classic Southhampton Stainless Steel Watch With Multi-Color Striped Band, Domire Fashion Accessories Tria

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [40]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5, learning_method = 'online')

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA requires integer values
tfidf_model_lda = CountVectorizer(max_df=0.99, max_features=500,
                                 min_df=0.01, stop_words='english',
                                 tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix_lda = tfidf_model_lda.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix_lda.shape[0]) + \
      " reviews and " + str(tfidf_matrix_lda.shape[1]) + " terms.")

In total, there are 1000 reviews and 245 terms.


In [42]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix_lda)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.02566481 0.02529649 0.02527951 0.02517366 0.89858553]
 [0.0511853  0.05145197 0.05108718 0.05090363 0.79537192]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10001704 0.10020281 0.10000836 0.5997716  0.10000019]
 [0.0515304  0.05127299 0.05203873 0.79430507 0.05085281]
 [0.39456232 0.03386602 0.5017883  0.03506509 0.03471828]]


In [43]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 245)
[[28.71404877 62.04122846 11.6009853  ...  2.16820733 34.59130007
  33.45384898]
 [ 6.87121777 54.39129098  0.20762923 ...  3.16519487  6.33981786
   6.88823582]
 [12.13376266 74.99860162  4.15882546 ...  5.34748972 24.34861302
   8.57270218]
 [ 0.23286589  9.82746105  0.20622895 ...  0.21179069  5.61276833
   4.20898919]
 [ 0.20467288 31.24976881  0.20520757 ...  7.01137233  0.21051725
   2.40555362]]


In [44]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.03,0.03,0.03,0.03,0.90,4
Doc1,0.05,0.05,0.05,0.05,0.80,4
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.69,0.03,0.03,0.22,0.03,0
Doc4,0.21,0.34,0.26,0.15,0.05,1
Doc5,0.04,0.04,0.61,0.04,0.27,2
Doc6,0.03,0.03,0.66,0.03,0.25,2
Doc7,0.69,0.23,0.03,0.03,0.03,0
Doc8,0.47,0.01,0.35,0.01,0.16,0
Doc9,0.02,0.68,0.26,0.02,0.02,1


In [45]:
df_document_topic['topic'].value_counts().to_frame()

,topic
2,309
0,247
3,207
4,146
1,91


In [46]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model_lda.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[28.71404877 62.04122846 11.6009853  ...  2.16820733 34.59130007
  33.45384898]
 [ 6.87121777 54.39129098  0.20762923 ...  3.16519487  6.33981786
   6.88823582]
 [12.13376266 74.99860162  4.15882546 ...  5.34748972 24.34861302
   8.57270218]
 [ 0.23286589  9.82746105  0.20622895 ...  0.21179069  5.61276833
   4.20898919]
 [ 0.20467288 31.24976881  0.20520757 ...  7.01137233  0.21051725
   2.40555362]]


,'m,'s,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,arm,arriv,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,br,bracelet,brand,break,bright,broke,button,buy,ca,came,case,...,star,start,stop,strap,sturdi,style,stylish,super,sure,surpris,swim,tell,thank,thing,think,thought,time,timex,tini,tri,turn,use,valu,ve,want,watch,water,way,wear,week,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,28.714049,62.041228,11.600985,0.224514,0.789738,3.146713,16.937016,0.364384,10.132970,0.388229,0.216389,18.973860,27.964163,15.477255,9.169133,0.207281,0.202679,8.872517,59.124970,37.711247,2.574347,9.584746,0.230117,2.460398,0.211445,19.352017,0.291989,13.251935,7.128157,0.407689,6.671414,5.122584,8.276330,0.211860,25.246279,9.257860,35.627007,10.718920,20.120038,18.234547,...,12.012148,12.938013,21.646086,13.507471,4.363207,25.148414,4.545219,0.202052,8.834158,3.191205,0.376787,0.327518,0.226285,5.568542,20.450204,1.666367,62.213252,11.812666,8.421268,11.279352,0.209455,51.807201,0.209838,21.083611,3.424189,275.756783,9.251730,7.054727,29.049877,48.183128,1.389133,7.084086,0.204351,0.203855,0.219773,73.126109,7.994701,2.168207,34.591300,33.453849
Topic1,6.871218,54.391291,0.207629,1.285268,6.400368,0.207324,0.603292,0.612484,0.202341,6.182658,2.500387,1.489537,0.695355,0.211615,0.215187,2.473051,0.202674,0.219522,41.863540,8.514653,12.523155,1.732576,0.228993,0.902116,0.305019,0.218679,1.003957,4.538785,0.760015,246.767034,0.327277,0.261927,2.424804,10.032405,0.203233,0.364120,5.271621,4.874949,0.204599,9.548949,...,3.061307,2.020381,6.097728,8.610974,0.204763,0.244723,0.323946,0.202152,9.152534,0.205344,0.206890,4.332324,3.883744,0.259339,1.766131,0.210185,58.161067,9.049911,0.350978,10.053293,9.992910,13.384748,0.337429,18.235064,4.861979,149.847078,7.904102,1.266828,11.602782,0.507609,0.343278,3.924033,0.204950,13.476561,3.490512,19.773035,0.310998,3.165195,6.339818,6.888236
Topic2,12.133763,74.998602,4.158825,0.210789,5.391043,5.769921,6.221767,0.748446,3.036295,4.358006,1.701319,5.830477,0.246275,0.209798,6.860354,14.468424,16.746868,8.875655,77.218280,0.879009,14.674506,2.173261,29.328324,41.608412,0.204557,6.256338,28.579591,11.605621,3.569822,0.277166,3.476246,3.800517,4.124930,0.206435,0.204073,0.216280,14.242572,9.908323,6.614281,0.933845,...,16.076648,0.265940,0.205723,25.974147,6.949813,0.222879,6.999165,9.880309,2.398454,9.068879,5.198321,0.322360,0.226270,12.288612,13.791241,10.069260,12.979318,0.210248,0.208743,0.222250,3.915666,9.425212,13.016393,3.226023,11.821912,389.604294,2.844951,8.672662,42.512737,0.375305,5.407430,2.103923,8.645074,0.210451,7.375849,3.690754,2.731727,5.347490,24.348613,8.572702
Topic3,0.232866,9.827461,0.206229,0.277732,0.263165,0.275684,0.241874,7.618405,0.204635,0.223847,15.868448,0.499334,0.207177,0.205399,5.697152,0.202023,0.205683,1.413780,7.649098,0.208973,2.530461,0.220092,0.205445,13.490270,0.210040,0.266832,0.205176,4.869252,0.211395,0.226958,0.745790,2.874910,0.249888,0.204188,0.206015,1.663550,5.537115,0.905230,1.111196,2.361956,...,0.214472,0.204977,0.246744,0.337138,0.211676,0.214768,0.726032,4.726362,0.937804,0.231152,0.205090,15.703105,13.176123,0.217530,1.256142,0.203229,33.020171,0.568946,0.202570,0.202021,0.201718,8.167311,8.323607,0.254208,16.821870,59.779802,0.205666,0.209820,0.509131,0.216283,0.300223,0.437933,0.202263,0.203059,0.825083,34.935741,0.203234,0.211791,5.612768,4.208989
Topic4,0.204673,31.249769,0.205208,15.437844,0.204357,7.063655,0.516519,8.210251,0.204441,2.603911,0.222238,0.215628,0.203429,0.203853,0.215902,0.205643,7.097683,6.087612,0.223531,0.209486,47.242329,3.723548,0.204790,3.497667,23.567763,0.206521,0.206559,14.930786,3.999537,0.216121,3.193431,1.241167,0.204926,0.254032,0.202366,5.699127,0.210309,0.207091,5.398223,0.271183,...,0.202413,0.203250,0.206404,0.203238,0.203614,6.050690,8.811572,0.213612,0.203328,0.224837,8.159429,2.907443,5.384003,6.643872,0.203794,4.529508,23.704877,0.242303,4.7

In [47]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model_lda, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,watch,n't,work,time,'s,band,fit,use,week,purchas,expect,month,batteri,order,like
Topic 1,br,watch,time,'s,day,band,n't,hand,set,date,look,second,design,size,work
Topic 2,watch,look,great,nice,band,'s,like,n't,realli,color,price,wear,big,cheap,light
Topic 3,good,look,watch,qualiti,excel,nice,like,price,work,time,product,great,far,need,want
Topic 4,love,watch,beauti,perfect,read,'s,wear,easi,gift,time,bit,light,compliment,littl,face
